In [118]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import sys,random
#def main(argv):
train = pd.read_csv('train.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader','atm_id']
train.drop(columns, inplace=True, axis=1)
train = train.filter(['target','distinct_days_gr4'
    ,'avg_cdm_delivery_count_distinct_day_gr4'
    ,'avg_session_error_count_gr5'
    ,'avg_cdm_dispense_amount_gr4'
    ,'avg_cdm_dispense_amount_distinct_day_gr4'
    ,'avg_cdm_delivery_status_error'
    ,'hours_since_last_cdm_error_gr4'
    ,'avg_session_error_count_distinct_day_gr3'
    ,'avg_card_capture_count_distinct_day_prev1'
    ,'hours_since_last_cdm_error_gr3'
    ,'distinct_days_gr2'
    ,'cdm_error_by_day_gr2'
    ,'avg_cdm_dispense_amount_distinct_day_gr2'
    ,'last_error_cdm_relative_gr2'
    ,'avg_cdm_error_gr2'
    ,'avg_cdm_dispense_amount_gr2'
    ,'cdm_error_by_day_gr1'
    ,'avg_front_prepare_count_distinct_day_gr1'
    ,'avg_cdm_error_gr1'
    ,'avg_bna_return_count_distinct_day_gr1'
    ,'cdm_error_by_disticnt_day_gr1'
    ,'distinct_days_prev1'
    ,'avg_cdm_dispense_amount_gr1'
    ,'avg_reply_count_distinct_day_gr1'
    ,'distinct_days_prev5'
    ,'status_to_transactions_prev5'
    ,'avg_cdm_dispense_amount_distinct_day_prev5'
    ,'avg_card_capture_count_distinct_day_prev5'
    ,'avg_front_prepare_count_prev5'
    ,'hours_since_last_cdm_error'
    ,'transactions_count_prev5'
    ,'status_to_transactions_prev4'
    ,'cdm_error_by_disticnt_day_prev4'
    ,'transactions_count_prev4'
    ,'avg_cdm_dispense_amount_prev4'
    ,'related_to_status_prev4'
    ,'distinct_days_prev3'
    ,'last_error_cdm_relative_prev3'
    ,'avg_front_prepare_count_prev3'
    ,'cdm_error_by_disticnt_day_prev3'
    ,'hours_since_last_cdm_error_prev3'
    ,'distinct_days_prev2'
    ,'avg_cdm_dispense_amount_distinct_day_gr5'
    ,'status_to_transactions_prev2'
    ,'avg_card_number_prev2'
    ,'avg_cdm_error_prev2'
    ,'avg_cdm_dispense_status_null_prev2'
    ,'avg_dispence_currency_0_gr1'
    ,'hours_since_last_cdm_error_prev2'
    ,'avg_cdm_delivery_status_not_taken_prev1'
    ,'cdm_label_gr1'
    ,'avg_card_number_gr5'
    ,'status_to_transactions_prev1'
    ,'avg_cdm_delivery_status_error_prev1'
    ,'avg_front_prepare_count_distinct_day_prev1'
    ,'avg_cdm_error_prev1'
    ,'avg_cdm_dispense_status_error_prev1'
    ,'cdm_error_by_disticnt_day_prev1'
    ,'transactions_count_prev1'
    ,'avg_receipt_confirm_count_distinct_day_prev5'
    ,'avg_request_count_prev1'
    ,'avg_receipt_confirm_count_prev1'
    ,'hours_since_last_cdm_error_prev1'
    ,'distinct_days'
    ,'status_to_transactions'
    ,'avg_card_number_prev5'
    ,'age'
    ,'related_to_transactions'
    ,'last_error_cdm_relative'
    ,'avg_cdm_error'
    ,'distinct_days_gr5'
    ,'avg_session_error_count_distinct_day_prev5'
    ,'cdm_error_by_disticnt_day'
    ])

#train['bmi'] = pd.qcut(train.distinct_days_prev1/(train.distinct_days_prev1/100)**2, 25, labels=False)
#train['age2'] = np.where(train['age']>=50, 1, 0)

y = train.target.values
train.drop(['target'], inplace=True, axis=1)
x = normalize(train.values)
#x = train.values

x, x_test, y, y_test = train_test_split(x, y, test_size=0.2)

y=np.asarray(y)

param_grid = {
        'depth':[3],
        'n_estimators': [22],
        'learning_rate': [0.03],
        'min_data_in_leaf': [60],
        'col_sample_rate' : [0.3]
        }

estimator = lightgbm.LGBMClassifier(
         boosting_type='dart',  
         objective='binary',
         metric=['auc'],
         penalty='l2'
        )

gridsearch = GridSearchCV(estimator, param_grid, scoring='f1',
        cv=5)

model=gridsearch.fit(x, y,
        eval_set=[(x_test, y_test)],
        #eval_metric=['auc'],
        early_stopping_rounds=3
        )

print('Best score: %0.3f' % gridsearch.best_score_)
print('Best parameters set:')
best_parameters = gridsearch.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
        
predictions = model.predict(x_test)

print('Validaton:', f1_score(y_test, predictions))

#if __name__ == '__main__':
#    main(sys.argv[1:])

#print('Save model...')
# save model to file
# AUC 0.67 - 70 
# AUC 0.65 - 65
#model.save_model('model.txt')

[1]	valid_0's binary_logloss: 0.690082
Training until validation scores don't improve for 3 rounds.
[2]	valid_0's binary_logloss: 0.686887
[3]	valid_0's binary_logloss: 0.68431
[4]	valid_0's binary_logloss: 0.681311
[5]	valid_0's binary_logloss: 0.678935
[6]	valid_0's binary_logloss: 0.676379
[7]	valid_0's binary_logloss: 0.674306
[8]	valid_0's binary_logloss: 0.675217
[9]	valid_0's binary_logloss: 0.672781
[10]	valid_0's binary_logloss: 0.670804
[11]	valid_0's binary_logloss: 0.668896
[12]	valid_0's binary_logloss: 0.669556
[13]	valid_0's binary_logloss: 0.667688
[14]	valid_0's binary_logloss: 0.665776
[15]	valid_0's binary_logloss: 0.664163
[16]	valid_0's binary_logloss: 0.662664
[17]	valid_0's binary_logloss: 0.661115
[18]	valid_0's binary_logloss: 0.659788
[19]	valid_0's binary_logloss: 0.658453
[20]	valid_0's binary_logloss: 0.656951
[21]	valid_0's binary_logloss: 0.657324
[22]	valid_0's binary_logloss: 0.656039
Did not meet early stopping. Best iteration is:
[22]	valid_0's binary

In [87]:
#
# Create a submission
#
test = pd.read_csv('test.csv',sep=';')
ids = test['atm_id'].values
columns = ['cash_in_out','display_type','scanner_code_reader','atm_id']
test.drop(columns, inplace=True, axis=1)
test = test.filter(['target','distinct_days_prev1','distinct_days_prev3','avg_cdm_error_gr1'
                      ,'distinct_days_prev2','avg_cdm_error_gr2','last_error_cdm_relative_gr2'
                      ,'avg_cdm_error','avg_cdm_error_prev1'
                     ,'cdm_error_by_day_gr2','cdm_label_gr1','avg_cdm_delivery_status_error'
                      ,'cdm_error_by_disticnt_day_gr1','distinct_days_prev5','transactions_count_prev1'
                      ,'avg_reply_count_distinct_day_gr1','status_to_transactions','avg_cdm_dispense_amount_gr4'
                      ,'avg_front_prepare_count_distinct_day_prev1','last_error_cdm_relative_prev4'
                      ,'avg_front_prepare_count_distinct_day_gr1'
                      ,'avg_card_number_prev5','avg_cdm_error_prev2','avg_front_prepare_count_prev5'])
x = test.values
#submission = pd.read_csv('out.csv')

#submission.drop('ATM_ID', inplace=True, axis=1)

y = model.predict(x)
y = [1 if i >=np.mean(y) else 0 for i in y]
output = pd.DataFrame({'ATM_ID': ids, 'PREDICT': y})
output.to_csv("out.csv", index=False)

In [114]:
predictions

array([0, 1, 0, ..., 1, 1, 1])